# Agents as Tools with Strands Agents


"Agents as Tools" is an architectural pattern in AI systems where specialized AI agents are wrapped as callable functions (tools) that can be used by other agents. This creates a hierarchical structure where:

1. A primary "orchestrator" agent handles user interaction and determines which specialized agent to call

2. Specialized "tool agents" perform domain-specific tasks when called by the orchestrator

This approach mimics human team dynamics, where a manager coordinates specialists, each bringing unique expertise to solve complex problems. Rather than a single agent trying to handle everything, tasks are delegated to the most appropriate specialized agent.



## Key Benefits and Core Principles

The "Agents as Tools" pattern offers several advantages:

- Separation of concerns: Each agent has a focused area of responsibility, making the system easier to understand and maintain
- Hierarchical delegation: The orchestrator decides which specialist to invoke, creating a clear chain of command
- Modular architecture: Specialists can be added, removed, or modified independently without affecting the entire system
- Improved performance: Each agent can have tailored system prompts and tools optimized for its specific task


In [1]:
!pip install -r requirements.txt

In [2]:
import os

from strands import Agent, tool
from strands_tools import file_write


In this module we will be creating an orchestrator based multi-agent workflow. 

<div style="text-align:left">
    <img src="images/architecture.png" width="75%" />
</div>

We will also explore `use_llm` which allows use to create nested agents.

## Research Agent

Lets first create a basic reasearch assistant with http_request tool. 

In [3]:
RESEARCH_ASSISTANT_PROMPT = """You are a specialized research assistant. Focus only on providing
factual, well-sourced information in response to research questions.
Always cite your sources when possible."""

In [4]:
research_agent = Agent(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    system_prompt=RESEARCH_ASSISTANT_PROMPT,
    # tools=[http_request]  # Here you can enable an agentic ai search tool
)

query = "Overview of Amazon Bedrock and its features"
# Call the agent and return its response
response = research_agent(query)

# Amazon Bedrock Overview

Amazon Bedrock is a fully managed service that provides access to high-performing foundation models (FMs) through a unified API. Launched in September 2023, it allows organizations to build and scale generative AI applications without extensive AI expertise.

## Key Features

### Foundation Model Access
- Offers a range of foundation models from Amazon and leading AI companies including:
  - Amazon Titan models (Text, Embeddings, Multimodal)
  - Anthropic (Claude, Claude Instant, Claude 3)
  - Meta (Llama 2, Llama 3)
  - AI21 Labs (Jurassic-2)
  - Stability AI (Stable Diffusion)
  - Cohere (Command, Embed)
- Single API access to all models for easy comparison and deployment[1][2]

### Customization Capabilities
- Model customization through fine-tuning using your own data
- Continued pre-training option for deeper customization
- Custom model evaluation to measure performance improvements[1][3]

### Enterprise Features
- Serverless experience with automatic s

Now we can wrap this agent as a tool. Allowing other agents to interact with it. 

####  Best Practices for Agent as Tools

When implementing the "Agents as Tools" pattern with Strands Agents:

1. Clear tool documentation: Write descriptive docstrings that explain the agent's expertise
2. Focused system prompts: Keep each specialized agent tightly focused on its domain
3. Proper response handling: Use consistent patterns to extract and format responses
4. Tool selection guidance: Give the orchestrator clear criteria for when to use each specialized agen

In [5]:
@tool
def research_assistant(query: str) -> str:
    """
    Process and respond to research-related queries.

    Args:
        query: A research question requiring factual information

    Returns:
        A detailed research answer with citations
    """
    try:
        # Strands agents makes it easy to create a specialized agent
        research_agent = Agent(
            model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
            system_prompt=RESEARCH_ASSISTANT_PROMPT,
        )

        # Call the agent and return its response
        response = research_agent(query)
        return str(response)
    except Exception as e:
        return f"Error in research assistant: {str(e)}"

Now lets follow the best practices and create `product_recommendation_assistant`, `trip_planning_assistant`, and `orchestrator` agent.

### Product Recommendation Assistant

In [6]:
@tool
def product_recommendation_assistant(query: str) -> str:
    """
    Handle product recommendation queries by suggesting appropriate products.

    Args:
        query: A product inquiry with user preferences

    Returns:
        Personalized product recommendations with reasoning
    """
    try:
        product_agent = Agent(
            model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",  # Optional: Specify the model ID
            system_prompt="""You are a specialized product recommendation assistant.
            Provide personalized product suggestions based on user preferences. Always cite your sources.""",
        )
        # Call the agent and return its response
        response = product_agent(query)

        return str(response)
    except Exception as e:
        return f"Error in product recommendation: {str(e)}"

In [7]:
product_recommendation_assistant("Product recommendations for flying cars")

# Product Recommendations for Flying Cars

While true flying cars aren't yet commercially available for consumer purchase, I can recommend some products related to this emerging technology:

## Current Options

1. **PAL-V Liberty** - The first commercially available flying car with road and air certification processes underway. It's essentially a gyrocopter that can also drive on roads. (~$400,000-600,000)

2. **Terrafugia Transition** - A roadable aircraft that's been in development for years, with reservations available. (~$300,000)

3. **AirCar by Klein Vision** - Recently completed a 35-minute flight between international airports in Slovakia. Still in certification process.

## Related Products

1. **Advanced drones** like the DJI Mavic series ($1,000-2,000) for those interested in aerial technology

2. **Flight simulators** such as X-Plane 12 or Microsoft Flight Simulator for PC to experience virtual flying ($60-80)

3. **RC flying car toys** like the SYMA X9 Flying Car ($40-60) 

"# Product Recommendations for Flying Cars\n\nWhile true flying cars aren't yet commercially available for consumer purchase, I can recommend some products related to this emerging technology:\n\n## Current Options\n\n1. **PAL-V Liberty** - The first commercially available flying car with road and air certification processes underway. It's essentially a gyrocopter that can also drive on roads. (~$400,000-600,000)\n\n2. **Terrafugia Transition** - A roadable aircraft that's been in development for years, with reservations available. (~$300,000)\n\n3. **AirCar by Klein Vision** - Recently completed a 35-minute flight between international airports in Slovakia. Still in certification process.\n\n## Related Products\n\n1. **Advanced drones** like the DJI Mavic series ($1,000-2,000) for those interested in aerial technology\n\n2. **Flight simulators** such as X-Plane 12 or Microsoft Flight Simulator for PC to experience virtual flying ($60-80)\n\n3. **RC flying car toys** like the SYMA X9 F

### Trip Planning Assistant

In [8]:
@tool
def trip_planning_assistant(query: str) -> str:
    """
    Create travel itineraries and provide travel advice.

    Args:
        query: A travel planning request with destination and preferences

    Returns:
        A detailed travel itinerary or travel advice
    """
    try:
        travel_agent = Agent(
            model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",  # Optional: Specify the model ID
            system_prompt="""You are a specialized travel planning assistant.
            Create detailed travel itineraries based on user preferences.""",
        )
        # Call the agent and return its response
        response = travel_agent(query)

        return str(response)
    except Exception as e:
        return f"Error in trip planning: {str(e)}"

### Orchestrator Agent

In [9]:
# Define orchestrator system prompt with clear tool selection guidance
MAIN_SYSTEM_PROMPT = """
You are an assistant that routes queries to specialized agents:
- For research questions and factual information → Use the research_assistant tool
- For product recommendations and shopping advice → Use the product_recommendation_assistant tool
- For travel planning and itineraries → Use the trip_planning_assistant tool
- For simple questions not requiring specialized knowledge → Answer directly

Always select the most appropriate tool based on the user's query.
"""

In [10]:
# Strands Agents allows easy integration of agent tools
orchestrator = Agent(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",  # Optional: Specify the model ID
    system_prompt=MAIN_SYSTEM_PROMPT,
    tools=[
        research_assistant,
        product_recommendation_assistant,
        trip_planning_assistant,
        file_write,
    ],
)

In [11]:
# Example: E-commerce Customer Service System
customer_query = (
    "I'm looking for hiking boots. Write the final response to current directory."
)

os.environ["BYPASS_TOOL_CONSENT"] = "true"

# The orchestrator automatically determines this requires multiple specialized agents
response = orchestrator(customer_query)

I'd be happy to help you find hiking boots. This sounds like a product recommendation query, and I'll also need to save the response to a file as requested.
Tool #1: product_recommendation_assistant
# Hiking Boot Recommendations

Based on your interest in hiking boots, here are some well-regarded options to consider:

## Popular Hiking Boot Options

1. **Salomon Quest 4 GTX** - Excellent ankle support and waterproofing, ideal for rugged terrain and backpacking
2. **Merrell Moab 3 Mid** - Known for out-of-the-box comfort and versatility, good for moderate trails
3. **Keen Targhee III** - Wider toe box, good for day hikes and light backpacking
4. **Hoka Anacapa Mid GTX** - Lightweight with cushioned soles, great for comfort on long hikes
5. **Danner Mountain 600** - Classic style with modern technology, good for mixed terrain

To make a better recommendation, I'd need to know more about:
- Your hiking experience level
- Typical terrain and conditions
- Your budget range
- Any specific fo

╔════════ File Write Operation ═════════╗
║                                       ║
║ Path: hiking_boot_recommendations.txt ║
║ Size: 927 characters                  ║
║                                       ║
╚═══════════════════════════════════════╝

╔══════════════════════ Write Successful ══════════════════════╗
║ File written successfully to hiking_boot_recommendations.txt ║
╚══════════════════════════════════════════════════════════════╝

I've found some hiking boot recommendations for you and saved them to a file named "hiking_boot_recommendations.txt" in the current directory.

The recommendations include popular options like the Salomon Quest 4 GTX, Merrell Moab 3 Mid, Keen Targhee III, Hoka Anacapa Mid GTX, and Danner Mountain 600. Each has different strengths depending on the type of hiking you plan to do.

If you'd like more personalized recommendations, I'd be happy to help if you can provide additional information about your hiking experience, typical terrain, budget, and any specific foot needs.

Lets look at the messages of the orchestrator. Here you can see the agent decided to use the sub-agent as tool

In [12]:
orchestrator.messages

[{'role': 'user',
  'content': [{'text': "I'm looking for hiking boots. Write the final response to current directory."}]},
 {'role': 'assistant',
  'content': [{'text': "I'd be happy to help you find hiking boots. This sounds like a product recommendation query, and I'll also need to save the response to a file as requested."},
   {'toolUse': {'toolUseId': 'tooluse_loA2Y9iSTP-F7S6Hq7tNQw',
     'name': 'product_recommendation_assistant',
     'input': {'query': "I'm looking for hiking boots. What are good options to consider?"}}}]},
 {'role': 'user',
  'content': [{'toolResult': {'toolUseId': 'tooluse_loA2Y9iSTP-F7S6Hq7tNQw',
     'status': 'success',
     'content': [{'text': "# Hiking Boot Recommendations\n\nBased on your interest in hiking boots, here are some well-regarded options to consider:\n\n## Popular Hiking Boot Options\n\n1. **Salomon Quest 4 GTX** - Excellent ankle support and waterproofing, ideal for rugged terrain and backpacking\n2. **Merrell Moab 3 Mid** - Known for o

In [13]:
customer_query = "Can you help me plan my trip to Patagonia"

response = orchestrator(customer_query)

I'd be happy to help you plan a trip to Patagonia! This is a perfect query for our trip planning assistant. To provide you with a useful itinerary, I'll use the travel planning tool.
Tool #3: trip_planning_assistant
# Patagonia Travel Itinerary

I'd be happy to help you plan an amazing Patagonia adventure! This region offers spectacular mountains, glaciers, and wildlife across both Argentina and Chile.

## Recommended Itinerary (14 Days)

### Days 1-3: El Calafate, Argentina
- **Day 1**: Arrival, settle in, explore town
- **Day 2**: Full-day excursion to Perito Moreno Glacier
- **Day 3**: Los Glaciares National Park boat tour

### Days 4-7: El Chaltén, Argentina
- **Day 4**: Travel to El Chaltén, afternoon short hike
- **Day 5**: Laguna de los Tres hike (Mount Fitz Roy viewpoint)
- **Day 6**: Laguna Torre trek
- **Day 7**: Rest day or Loma del Pliegue Tumbado hike

### Days 8-11: Torres del Paine, Chile
- **Day 8**: Travel to Torres del Paine National Park
- **Days 9-11**: W Trek highl

In [14]:
orchestrator.messages

[{'role': 'user',
  'content': [{'text': "I'm looking for hiking boots. Write the final response to current directory."}]},
 {'role': 'assistant',
  'content': [{'text': "I'd be happy to help you find hiking boots. This sounds like a product recommendation query, and I'll also need to save the response to a file as requested."},
   {'toolUse': {'toolUseId': 'tooluse_loA2Y9iSTP-F7S6Hq7tNQw',
     'name': 'product_recommendation_assistant',
     'input': {'query': "I'm looking for hiking boots. What are good options to consider?"}}}]},
 {'role': 'user',
  'content': [{'toolResult': {'toolUseId': 'tooluse_loA2Y9iSTP-F7S6Hq7tNQw',
     'status': 'success',
     'content': [{'text': "# Hiking Boot Recommendations\n\nBased on your interest in hiking boots, here are some well-regarded options to consider:\n\n## Popular Hiking Boot Options\n\n1. **Salomon Quest 4 GTX** - Excellent ankle support and waterproofing, ideal for rugged terrain and backpacking\n2. **Merrell Moab 3 Mid** - Known for o

### Calling multiple agents 

In [15]:
orchestrator.messages = []

In [16]:
query = "Can you do a research on spain? Also help me plan a 7 day trip."

orchestrator(query)

I'd be happy to help with both your research on Spain and planning a 7-day trip there. I'll use specialized tools for each part of your request.

First, let me gather some general research information about Spain:
Tool #4: research_assistant
# Comprehensive Overview of Spain

## Geography and Demographics

Spain occupies most of the Iberian Peninsula in southwestern Europe, covering an area of about 505,990 square kilometers. It borders Portugal to the west, France and Andorra to the north, and Gibraltar to the south[1].

Key geographic features include:
- The Meseta Central (central plateau)
- Several mountain ranges including the Pyrenees in the north and Sierra Nevada in the south
- Major rivers: Ebro, Duero, Tajo, Guadiana, and Guadalquivir
- The Balearic Islands in the Mediterranean and the Canary Islands in the Atlantic

Spain has a population of approximately 47 million people, with Madrid (3.3 million) as its capital and largest city, followed by Barcelona (1.6 million)[2].

##

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': "## Summary of Spain\n\nBased on the research, Spain is a diverse and culturally rich country in southwestern Europe with a fascinating history spanning from ancient pre-Roman civilizations through its powerful imperial era to modern democracy. \n\nSome key highlights about Spain:\n\n1. **Geography**: Spain occupies most of the Iberian Peninsula with diverse landscapes including mountains, plateaus, and coastlines. It includes the Balearic Islands in the Mediterranean and the Canary Islands in the Atlantic.\n\n2. **History**: Spain's rich history includes Roman rule, Islamic influence during Al-Andalus, the Reconquista, its powerful imperial period after 1492, and its modern democratic transition following Franco's dictatorship.\n\n3. **Culture**: Spain has an incredibly rich cultural heritage featuring:\n   - Renowned art from masters like Velázquez, Goya, Picasso, and Dalí\n   - Literary contributi

Behind the scenes, the orchestrator will:
1. First call the `research_assistant`
2. Then call `trip_planning_assistant`
3. Combine these specialized responses into a cohesive answer that addresses both the queries

### Sequential Agent Communication Pattern


The agent tool can also combine multiple agents together. In this example we will provide output of `research_agent` to `summary_agent` and return the summarized response.

In [17]:
 # define the user query
topic = "generative Ai"
# Create a research agent
research_agent = Agent(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    system_prompt=RESEARCH_ASSISTANT_PROMPT,
)
# Create a summarization agent
summary_agent = Agent(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    system_prompt="""
    You are a summarization specialist focused on distilling complex information into clear, concise summaries.
    Your primary goal is to take detailed information and extract the key points, main arguments, and critical data.
    You should maintain the accuracy of the original content while making it more digestible.
    Focus on clarity, brevity, and highlighting the most important aspects of the information.
    """,
)

print("Multiple agents created successfully!")
print(f"\n🔍 RESEARCH AGENT working on: {topic}\n") 
try:
    # Agent 1: Invoke research agent
    research_response = research_agent(
        f"Please gather comprehensive information about {topic}."
    )
    research_text = research_response.message.content[0]["text"]
    print("\n✂️ SUMMARY AGENT distilling the research\n")
    
    # Agent 2: Ask the summary agent to create a concise summary
    summary_response = summary_agent(
        f"Please create a concise summary of this research: {research_text}"
    )
    summary_text = summary_response.message.content[0]["text"]
    
    print(summary_text)
except Exception as e:
    print(f"Error in research assistant: {str(e)}")

Multiple agents created successfully!

🔍 RESEARCH AGENT working on: generative Ai

# Generative AI: A Comprehensive Overview

Generative AI refers to artificial intelligence systems capable of generating new content such as text, images, audio, video, code, and 3D models that resemble but are distinct from their training data.

## Core Technologies

### Foundation Models
Large-scale AI models trained on vast datasets that serve as the base for various applications:
- **Large Language Models (LLMs)**: Systems like GPT-4 (OpenAI), PaLM/Gemini (Google), Claude (Anthropic), and Llama (Meta) that generate human-like text[1][2].
- **Diffusion Models**: Power most modern image generation systems like DALL-E, Midjourney, and Stable Diffusion[3].
- **Multimodal Models**: Systems that work across different data types, like GPT-4V, Gemini, and Claude Opus, integrating text, images, and sometimes audio[2].

### Technical Foundations
- **Transformer Architecture**: The dominant neural network archi

## Congrats!

You've learned how to use agents as tools in Strands Agents to create more complex agentic applications